In [28]:
import numpy as np
import pymorton as pm
import ast
import math

In [29]:
# input files for operation
Rtreefile = input('please input Rtree file:')
Queryfile = input('please input Query file:')

please input Rtree file:Rtree.txt
please input Query file:Rqueries.txt


In [36]:
#read the input txt files
Wqueries = []
with open(Queryfile,'r') as file:
    for line in file:
        new_line = line.split(' ')
        new_line_float = list(map(float, new_line))
        Wqueries.append(new_line_float)


StrRtree = []
with open(Rtreefile,'r') as file:
    for line in file:
        StrRtree.append(line)
    
# convert string of the input Rtree file to list and keep the list hierarchy
def convert_list(lst):
    newlist = []
    for i in lst:
        if isinstance(i, list):
            newlist.append(convert_list(i))
        else:
            newlist.append(i)
    return newlist
    
Rtree = []
for string in StrRtree:
    singlelist = ast.literal_eval(string)
    newlist = convert_list(singlelist)
    Rtree.append(newlist)

In [37]:
# recount the nodes numbers and how many levels for the Rtree
# set the node capacity for the Rtree
nodecapacityM = 20
nodecapacitym = 0.4*nodecapacityM

#calculate the numbers of leaf node in the Rtree
leafnodeNo = sum(1 for nodes in Rtree if nodes[0] == 0)

#calculate the numbers and levels of nonleaf node part of the Rtree
nonleafnodeNo = math.ceil(leafnodeNo/nodecapacityM)
nonleafnode_level = [nonleafnodeNo]
while nonleafnodeNo > 1:
    nonleafnodeNo = math.ceil(nonleafnodeNo/nodecapacityM)
    nonleafnode_level.append(nonleafnodeNo)

Rtreelevel = len(nonleafnode_level)+1
NLN_level = nonleafnode_level[::-1]

In [38]:
#rebuild Rtree
NewRtree = []

for i in NLN_level:
    nonleafnode_list = Rtree[-i:]
    del Rtree[-i:]
    NewRtree.append(nonleafnode_list)
    
NewRtree.append(Rtree)

In [39]:
# range query function
def WQ_xycompare(j):
    xmin = j[1][0]
    xmax = j[1][1]
    ymin = j[1][2]
    ymax = j[1][3]
        
    if xmax< Wx_min or xmin > Wx_max or ymax < Wy_min or ymin > Wy_max:
        return
    else:
        return j[0]
    
def delNone(lst):
    newlist = [x for x in lst if x is not None]
    return newlist

In [40]:
# Scan Rtree function, iteratly build index of each level's intersection node with the range querie, and scan the next level's nodes basing on the index of upper level
def scan_rtree(lst):
    global index
    
    if len(lst) != 0:
        newindex = []
        for i in index:
            for j in lst[0][i][2]:
                newindex.append(WQ_xycompare(j))
        newindex = delNone(newindex)
            
    del lst[0]
    return newindex

In [42]:
# apply the list of range queries to the Rtree and print the result
WQid = 0
for wq in Wqueries:
    Wx_min = wq[0]
    Wx_max = wq[2]
    Wy_min = wq[1]
    Wy_max = wq[3]
    
    ScanRtree = NewRtree[:]
    
    index = [0]
    
    for i in range(len(NewRtree)):
        scan_index = scan_rtree(ScanRtree)
        index = scan_index
        
    print(str(WQid)+'('+str(len(index))+'): '+','.join(str(x) for x in index))
    WQid += 1

0(7): 2527,2712,8371,5042,7080,7656,7944
1(101): 7899,3508,1018,3394,8090,9349,3364,475,3999,764,2738,4606,9362,9303,6073,6389,3048,6714,5898,4990,3107,1980,5792,8019,1006,6294,7924,7923,6374,9987,3185,1044,3325,6902,9974,2294,8842,1506,8207,8475,666,8910,1277,7501,2249,5292,5499,4978,3507,8086,1440,4656,3044,6530,8970,3174,4727,817,7108,3430,9919,83,8594,6147,6256,5592,2864,2708,8367,1619,2008,3882,5037,2117,3140,151,2108,5597,7770,5872,3845,4073,7430,6442,420,164,532,6571,5916,1600,2359,8747,5304,8415,4734,5030,8865,5562,8335,1051,6493
2(98): 9039,4158,6710,6611,702,5290,6975,2428,1500,2735,2424,4379,9603,2385,4587,7593,3627,210,2136,5440,5696,6727,9591,8312,4027,7866,9268,3823,5111,5842,8771,4685,7759,8197,6999,1484,6921,8036,954,8863,7410,463,4058,6845,6607,486,125,240,9888,792,1915,9338,7828,4127,2662,1920,3533,6230,6027,3075,1014,7600,1595,7197,7964,9233,4394,3705,3801,2467,402,2070,8102,2308,7984,382,3371,7562,1304,926,1088,6765,720,5241,4406,5749,8283,5114,6900,1342,6198,6783,9

651(132): 9231,9506,8519,4628,4478,4796,3412,3010,3628,7857,7735,9872,4322,5934,2260,55,9549,969,6947,2462,7364,8529,4120,2592,6413,7393,8652,115,4619,2449,1742,8067,3035,717,4585,4028,6683,9617,1827,4615,5964,3566,7153,8366,4979,703,4576,3908,6923,4609,7669,2848,5417,4086,2085,8988,7210,5952,8520,9876,76,3419,1939,440,7204,8121,1291,2729,7803,8663,7998,3707,2682,8259,345,5990,7152,4999,1415,7749,2944,1690,7434,3356,3105,1202,687,1797,725,1681,6432,5514,4944,9354,5946,2817,8334,9537,4428,8822,112,6362,2801,6818,780,6190,7583,197,9428,5619,980,3924,2909,8527,5744,6747,4405,3444,3864,6212,5714,377,155,9319,7624,121,4590,9807,6901,7105,1554,3395
652(86): 182,88,1310,1625,4055,1835,135,79,3623,9421,1880,246,2888,8566,7040,5991,3171,8417,5160,7635,1898,1425,888,2317,8465,7542,5090,4146,1004,5587,2007,8034,5168,3464,9217,815,2132,1329,1019,8318,1325,3887,139,909,3568,2900,2099,9814,5348,673,8969,4279,9753,6709,1401,160,3327,2757,2585,3747,5701,8777,6254,5608,4724,9348,90,9258,2441,5391,5096,